<a href="https://colab.research.google.com/github/mathewpolonsky/Marketplace-Item-Matching/blob/main/Adding_Differerences2FullText.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -U sentence-transformers
!pip install evaluate
!pip install transformers[torch]

In [ ]:
import torch
from torch.utils.data import DataLoader

import numpy as np
from datetime import datetime

import pandas as pd

from transformers import AutoTokenizer, AutoModelForSequenceClassification, AutoModelForMaskedLM

from sentence_transformers import SentenceTransformer, losses, InputExample, evaluation, models, util

from sklearn.model_selection import train_test_split

import re

from difflib import Differ

from tqdm.notebook import tqdm

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!gdown 1N8fUo17cyn4UTSfuebL6RS3-tFb3nWe9
!unzip merged_t.zip

Downloading...
From: https://drive.google.com/uc?id=1N8fUo17cyn4UTSfuebL6RS3-tFb3nWe9
To: /content/merged_t.zip
100% 333M/333M [00:04<00:00, 67.8MB/s]
Archive:  merged_t.zip
  inflating: merged_test4nlp_w_diff.parquet  
  inflating: merged_train4nlp_w_diff.parquet  


In [ ]:
train = pd.read_parquet("merged_train4nlp_w_diff.parquet")
test = pd.read_parquet("merged_test4nlp_w_diff.parquet")

In [ ]:
train.target.value_counts()

0.0    171527
1.0    135013
Name: target, dtype: int64

In [ ]:
135013 / (171527 + 135013)

0.4404417041821622

In [ ]:
train

,target,variantid1,variantid2,full_text_1,full_text_2,text_difference
0,0.0,51197862,51198054,name: Удлинитель TDM Electric Люкс УЛ05В 5 м (...,name: Удлинитель TDM Electric Люкс УЛ05В 1.5 м...,Товар 1 | name: Удлинитель TDM Electric Люкс У...
1,0.0,51197862,51199884,name: Удлинитель TDM Electric Люкс УЛ05В 5 м (...,name: Удлинитель TDM Electric Люкс УЛ05В 3 м (...,Товар 1 | name: Удлинитель TDM Electric Люкс У...
2,1.0,53062686,536165289,name: Картридж лазерный Комус 729 (4370B002) ч...,name: Картридж лазерный Комус 729 (4368B002) п...,"Товар 1 | color: черный, чер\nТовар 2 | color:..."
3,1.0,53602615,587809782,name: Картридж лазерный Комус 729 (4368B002) п...,name: Картридж лазерный Комус 729 (4370B002) ч...,Товар 1 | color: пурпурный\nТовар 2 | color: ч...
4,1.0,53602615,615149925,name: Картридж лазерный Комус 729 (4368B002) п...,name: Картридж лазерный Комус 729 (4368B002) п...,Товар 1 | name: Картридж лазерный Комус 729 (4...
...,...,...,...,...,...,...
306535,0.0,810680230,820119986,"name: Комплект 4 шт, Картридж лазерный NV Prin...","name: Комплект 2 шт, Картридж лазерный NV Prin...","Товар 1 | name: Комплект 4 шт, Картридж лазерн..."
306536,0.0,812434186,815345877,name: Для iPhone 7 плюс 8 плюс военная броня г...,name: Для iPhone 7 8 военная броня корпус гибр...,Товар 1 | color: желтый\nТовар 2 | color: сере...
306537,0.0,815637954,817550808,name: Карта памяти 64 ГБ\ncat3: Карты памяти и...,name: Карта памяти 1 ТБ\ncat3: Карты памяти и ...,Товар 1 | name: Карта памяти 64 ГБ\nТовар 2 | ...
306538,0.0,817327230,822083612,"name: Смартфон Mate48 Pro.. 8/256 ГБ, зеленый\...","name: Смартфон Mate48 Pro.. 10/512 ГБ, зеленый...",Товар 1 | name: Смартфон Mate48 Pro.. 8/256 ГБ...


#### a lot of sega

In [ ]:
sega_count = train.apply(lambda x: (("Sega" in x["full_text_1"]) or ("Sega" in x["full_text_2"])), axis=1)

In [ ]:
train[sega_count]

,target,variantid1,variantid2,full_text_1,full_text_2,text_difference
894,1.0,547440723,739653409,name: Портативная игровая приставка Game Stick...,name: Портативная игровая приставка Game Stick...,Товар 1 | Бренд: Joshua\nТовар 2 | Бренд: Sega...
896,1.0,547440723,777589637,name: Портативная игровая приставка Game Stick...,name: Портативная игровая приставка Game Stick...,Товар 1 | Бренд: Joshua\nТовар 2 | Бренд: Sega...
897,1.0,534210560,777589637,name: Портативная игровая приставка Game Stick...,name: Портативная игровая приставка Game Stick...,Товар 1 | name: Портативная игровая приставка ...
1467,1.0,770483286,775499361,name: Портативная игровая приставка Game Stick...,name: Портативная игровая приставка Game Stick...,Товар 1 | Бренд: Sega\nТовар 2 | Бренд: Космос...
1468,1.0,750245586,775499361,name: Портативная игровая приставка Game Stick...,name: Портативная игровая приставка Game Stick...,Товар 1 | Беспроводные интерфейсы: Bluetooth\n...
...,...,...,...,...,...,...
302590,1.0,777588818,777593537,"name: игровая приставка Game Stick Lite ""PREMl...",name: Портативная игровая приставка Game Stick...,Товар 1 | name: игровая приставка Game Stick L...
303809,1.0,770483871,770483901,name: Портативная игровая приставка Game Stick...,name: Портативная игровая приставка Game Stick...,"Товар 1 | Доп. комплектация: 2 геймпада, Карта..."
304042,1.0,484829544,749053130,name: Игровая приставка 16 Bit Super Drive GTA...,name: Игровая приставка 16 bit Super Drive GTA...,"Товар 1 | color: pink, черный, green\nТовар 2 ..."
304204,1.0,761122831,761167902,name: Портативная игровая приставка Game Stick...,name: Портативная игровая приставка Game Stick...,Товар 1 | Бренд: Нет бренда\nТовар 2 | Бренд: ...


In [ ]:
sega_count.sum() / sega_count.shape[0]

0.039120506296078814

### text diff

In [ ]:
# remove every occurrence after fourth time
mask = train.groupby("text_difference").cumcount() > 5
mask = mask & (train.text_difference != "Товар 1 и Товар 2 одинаковы")

In [ ]:
train[mask].target.value_counts()

1.0    11876
0.0     1339
Name: target, dtype: int64

In [ ]:
train = train[~mask]

In [ ]:
train

,target,variantid1,variantid2,full_text_1,full_text_2,text_difference
0,0.0,51197862,51198054,name: Удлинитель TDM Electric Люкс УЛ05В 5 м (...,name: Удлинитель TDM Electric Люкс УЛ05В 1.5 м...,Товар 1 | name: Удлинитель TDM Electric Люкс У...
1,0.0,51197862,51199884,name: Удлинитель TDM Electric Люкс УЛ05В 5 м (...,name: Удлинитель TDM Electric Люкс УЛ05В 3 м (...,Товар 1 | name: Удлинитель TDM Electric Люкс У...
2,1.0,53062686,536165289,name: Картридж лазерный Комус 729 (4370B002) ч...,name: Картридж лазерный Комус 729 (4368B002) п...,"Товар 1 | color: черный, чер\nТовар 2 | color:..."
3,1.0,53602615,587809782,name: Картридж лазерный Комус 729 (4368B002) п...,name: Картридж лазерный Комус 729 (4370B002) ч...,Товар 1 | color: пурпурный\nТовар 2 | color: ч...
4,1.0,53602615,615149925,name: Картридж лазерный Комус 729 (4368B002) п...,name: Картридж лазерный Комус 729 (4368B002) п...,Товар 1 | name: Картридж лазерный Комус 729 (4...
...,...,...,...,...,...,...
306535,0.0,810680230,820119986,"name: Комплект 4 шт, Картридж лазерный NV Prin...","name: Комплект 2 шт, Картридж лазерный NV Prin...","Товар 1 | name: Комплект 4 шт, Картридж лазерн..."
306536,0.0,812434186,815345877,name: Для iPhone 7 плюс 8 плюс военная броня г...,name: Для iPhone 7 8 военная броня корпус гибр...,Товар 1 | color: желтый\nТовар 2 | color: сере...
306537,0.0,815637954,817550808,name: Карта памяти 64 ГБ\ncat3: Карты памяти и...,name: Карта памяти 1 ТБ\ncat3: Карты памяти и ...,Товар 1 | name: Карта памяти 64 ГБ\nТовар 2 | ...
306538,0.0,817327230,822083612,"name: Смартфон Mate48 Pro.. 8/256 ГБ, зеленый\...","name: Смартфон Mate48 Pro.. 10/512 ГБ, зеленый...",Товар 1 | name: Смартфон Mate48 Pro.. 8/256 ГБ...


In [ ]:
mask = (train.groupby("variantid1").cumcount() > 2) | (train.groupby("variantid2").cumcount() > 2)

In [ ]:
train[mask].target.value_counts()

0.0    7761
1.0    3453
Name: target, dtype: int64

In [ ]:
train = train[~mask]

In [ ]:
train.target.value_counts()

0.0    162427
1.0    119684
Name: target, dtype: int64

### remove more 0's

In [ ]:
mask_target0 = (train.target == 0)

In [ ]:
mask = (train.groupby("text_difference").cumcount() > 1) & (mask_target0)
# mask = mask & (train.text_difference != "Товар 1 и Товар 2 одинаковы")

In [ ]:
mask.sum()

4207

In [ ]:
train = train[~mask]

In [ ]:
mask_target0 = (train.target == 0)

In [ ]:
mask = ((train.groupby("variantid1").cumcount() > 1) | 
        (train.groupby("variantid2").cumcount() > 1)) & (mask_target0)

In [ ]:
mask.sum()

10888

In [ ]:
train = train[~mask]

In [ ]:
train.target.value_counts()

0.0    147332
1.0    119684
Name: target, dtype: int64

In [ ]:
119684 / (119684 + 147332)

0.44822782155376456

In [ ]:
train.shape

(267016, 6)

In [ ]:
267016 / (171527 + 135013)

0.8710641351862726

In [ ]:
ones = train[train.target == 1]
zeros = train[train.target == 0]

zeros = zeros.sample(ones.shape[0])

balanced = pd.concat([ones, zeros])

In [ ]:
balanced.target.value_counts()

1.0    119684
0.0    119684
Name: target, dtype: int64

In [ ]:
balanced.shape[0] / (171527 + 135013)

0.7808703594963137

### new diff

In [ ]:
def preprocess_full_text(text):
    ':'.join(i[:300] for i in text.split(':'))

    pattern = r'\n{0,1}[^:\n]+: '
    matches = [(m.group(0), (m.start(), m.end()-1)) for m in re.finditer(pattern, text)]

    keys, values = [], []
    for i in range(1, len(matches)):
        keys.append(matches[i-1][0])
        start, end = matches[i-1][1][1] + 1, matches[i][1][0]
        values.append(text[start:end])

    i += 1
    keys.append(matches[i-1][0])
    start = matches[i-1][1][1] + 1
    values.append(text[start:])

    # text_key_split = re.split("(\n{0,1}[^:\n]+: )", text)
    
    # name = text_key_split[0]
    # keys, values = text_key_split[1::2], text_key_split[2::2]

    have_color = False

    for i in range(min(4, len(keys))):
        if keys[i].strip() == "name:":
            keys[i] = "Имя :"
        
        elif keys[i].strip() == "color:":
            keys[i] = "Цвет :"
            have_color = True
        
        elif keys[i].strip() == "cat3:":
            keys[i] = "Категория 3 :"
        
        elif keys[i].strip() == "cat4:":
            keys[i] = "Категория 4 :"

    special_keys = [i.replace('\n', ' ') for i in keys[:3+have_color]]
    special_values = [i.replace('\n', ' ') for i in values[:3+have_color]]

    special = [f"{special_keys[i]} {special_values[i]}" for i in range(len(special_keys))]

    other_keys = [i.strip().replace('\n', ' ')[:-1] + " :" for i in keys[3+have_color:]]
    other_values = [i.strip().replace('\n', ' ') for i in values[3+have_color:]]

    other = [f"{other_keys[i]} {other_values[i]}" for i in range(len(other_keys))]

    return special + sorted(other)


differ = Differ()

def find_diff(text1, text2):
    text1, text2 = preprocess_full_text(text1), preprocess_full_text(text2)

    new_text1, new_text2, common = "", "", ""
    
    diff = differ.compare(text1, text2)
    
    for text in diff:
        if text:
            if text[0] == '-':
                new_text1 += f"(Отличается) {text[2:]}\n"

            elif text[0] == '+':
                new_text2 += f"(Отличается) {text[2:]}\n"

            elif text[0] == ' ':
                common += text[2:] + '\n'
    
    new_text1 += common
    new_text2 += common
    
    new_text1 = new_text1.strip()
    new_text2 = new_text2.strip()

    return new_text1, new_text2

In [ ]:
num = 0

print(find_diff(train["full_text_1"][num], train["full_text_2"][num])[0])
print('\n')
print(find_diff(train["full_text_1"][num], train["full_text_2"][num])[1])

(Отличается) Имя : Удлинитель TDM Electric Люкс УЛ05В 5 м (SQ1303-0138)
(Отличается) Длина кабеля питания, м : 5
Цвет : белый
Категория 3 : Сетевые фильтры, разветвители и удлинители
Категория 4 : Сетевой фильтр, удлинитель, разветвитель
Бренд : TDM Electric
Вес товара, г : 430
Вид кабеля : ПВС
Вид розеток : Евровилка
Входное напряжение, В : 250
Гарантийный срок : 3 года
Количество розеток, шт : 5
Комплектация : инструкция по эксплуатации
Макс. нагрузка, Вт : 3500
Макс. ток, А : 16
Номинальный ток, А : 16
Сечение жилы, кв.мм : 1
Стандарт защиты : IP20
Тип : Удлинитель бытовой
Цвет товара : белый
Число жил : 3
Электробезопасность : Заземление


(Отличается) Имя : Удлинитель TDM Electric Люкс УЛ05В 1.5 м (SQ1303-0136)
(Отличается) Длина кабеля питания, м : 1.5
Цвет : белый
Категория 3 : Сетевые фильтры, разветвители и удлинители
Категория 4 : Сетевой фильтр, удлинитель, разветвитель
Бренд : TDM Electric
Вес товара, г : 430
Вид кабеля : ПВС
Вид розеток : Евровилка
Входное напряжение, В : 

### make new diff

In [ ]:
num = 236712

print(find_diff(train["full_text_1"][num], train["full_text_2"][num])[0])
print('\n')
print(find_diff(train["full_text_1"][num], train["full_text_2"][num])[1])

(Отличается) Имя : RITMIX RH-533USB, Silver Гарнитура
(Отличается) Цвет : silver, черный, серебристый
(Отличается) Вес товара, г : 0.28
(Отличается) Длина кабеля, см : 240
(Отличается) Конструкция наушников : Полноразмерные
(Отличается) Цвет товара : серебристый, черный
Категория 3 : Наушники и гарнитуры
Категория 4 : Игровые наушники
Бренд : Ritmix
Макс. частота, Гц : 20000
Микрофон : Да
Мин. частота, Гц : 20
Разъем : USB
Сопротивление, Ом : 32
Страна-изготовитель : Китай
Тип : Игровые наушники
Тип подключения : Проводное


(Отличается) Имя : Игровые наушники Ritmix RH-533 USB Silver, серебристый
(Отличается) Цвет : silver, серебристый
(Отличается) Акустическое оформление : Закрытое
(Отличается) Вес товара, г : 154
(Отличается) Гарантийный срок : Гарантия от производителя
(Отличается) Диаметр динамика, мм : 30
(Отличается) Длина кабеля, см : 180
(Отличается) Защита от влаги : Нет
(Отличается) Кабель и штекер : Одностороннее подключение кабеля
(Отличается) Комплектация : Наушники, доку

In [ ]:
17953

In [ ]:
preprocess_full_text(train.iloc[744]["full_text_1"])

['Имя : Стабилизатор для телефона GimbalPro L7Bpro',
 'Цвет : серый',
 'Категория 3 : Штативы и головки',
 'Категория 4 : Монопод для селфи',
 'Бренд : WIWU',
 'Вес товара, г : 362',
 'Гарантийный срок : 6 месяцев',
 'Дистанционное управление : Кнопка на рукоятке (Bluetooth)',
 'Макс. нагрузка, г : 280',
 'Макс. ширина удерживаемого устройства, мм : 90',
 'Материал : Пластик',
 'Назначение : Универсальные',
 'Размеры, мм : 152x110x47',
 'Страна-изготовитель : Китай',
 'Тип : Стабилизатор для смартфона',
 'Цвет товара : серый']

In [ ]:
new_texts1, new_texts2 = [], []

for i in tqdm(range(test.shape[0])):
    text1, text2 = test.iloc[i]["full_text_1"], test.iloc[i]["full_text_2"]
    new_text1, new_text2 = find_diff(text1, text2)
    new_texts1.append(new_text1), new_texts2.append(new_text2)

  0%|          | 0/18084 [00:00<?, ?it/s]

In [ ]:
test["full_text_1_w_diff"] = new_texts1
test["full_text_2_w_diff"] = new_texts2

In [ ]:
test = test.drop(["full_text_1", "full_text_2", "text_difference"], axis=1)

In [ ]:
test.to_parquet("test_full_text_w_diff.parquet", index=False)

In [ ]:
new_texts1, new_texts2 = [], []

for i in tqdm(range(train.shape[0])):
    text1, text2 = train.iloc[i]["full_text_1"], train.iloc[i]["full_text_2"]
    new_text1, new_text2 = find_diff(text1, text2)
    new_texts1.append(new_text1), new_texts2.append(new_text2)

  0%|          | 0/267016 [00:00<?, ?it/s]

In [ ]:
train["full_text_1_w_diff"] = new_texts1
train["full_text_2_w_diff"] = new_texts2

In [ ]:
train

,target,variantid1,variantid2,full_text_1,full_text_2,text_difference,full_text_1_w_diff,full_text_2_w_diff
0,0.0,51197862,51198054,name: Удлинитель TDM Electric Люкс УЛ05В 5 м (...,name: Удлинитель TDM Electric Люкс УЛ05В 1.5 м...,Товар 1 | name: Удлинитель TDM Electric Люкс У...,(Отличается) Имя : Удлинитель TDM Electric Люк...,(Отличается) Имя : Удлинитель TDM Electric Люк...
1,0.0,51197862,51199884,name: Удлинитель TDM Electric Люкс УЛ05В 5 м (...,name: Удлинитель TDM Electric Люкс УЛ05В 3 м (...,Товар 1 | name: Удлинитель TDM Electric Люкс У...,(Отличается) Имя : Удлинитель TDM Electric Люк...,(Отличается) Имя : Удлинитель TDM Electric Люк...
2,1.0,53062686,536165289,name: Картридж лазерный Комус 729 (4370B002) ч...,name: Картридж лазерный Комус 729 (4368B002) п...,"Товар 1 | color: черный, чер\nТовар 2 | color:...",(Отличается) Имя : Картридж лазерный Комус 729...,(Отличается) Имя : Картридж лазерный Комус 729...
3,1.0,53602615,587809782,name: Картридж лазерный Комус 729 (4368B002) п...,name: Картридж лазерный Комус 729 (4370B002) ч...,Товар 1 | color: пурпурный\nТовар 2 | color: ч...,(Отличается) Имя : Картридж лазерный Комус 729...,(Отличается) Имя : Картридж лазерный Комус 729...
4,1.0,53602615,615149925,name: Картридж лазерный Комус 729 (4368B002) п...,name: Картридж лазерный Комус 729 (4368B002) п...,Товар 1 | name: Картридж лазерный Комус 729 (4...,(Отличается) Имя : Картридж лазерный Комус 729...,(Отличается) Имя : Картридж лазерный Комус 729...
...,...,...,...,...,...,...,...,...
306535,0.0,810680230,820119986,"name: Комплект 4 шт, Картридж лазерный NV Prin...","name: Комплект 2 шт, Картридж лазерный NV Prin...","Товар 1 | name: Комплект 4 шт, Картридж лазерн...","(Отличается) Имя : Комплект 4 шт, Картридж лаз...","(Отличается) Имя : Комплект 2 шт, Картридж лаз..."
306536,0.0,812434186,815345877,name: Для iPhone 7 плюс 8 плюс военная броня г...,name: Для iPhone 7 8 военная броня корпус гибр...,Товар 1 | color: желтый\nТовар 2 | color: сере...,(Отличается) Имя : Для iPhone 7 плюс 8 плюс во...,(Отличается) Имя : Для iPhone 7 8 военная брон...
306537,0.0,815637954,817550808,name: Карта памяти 64 ГБ\ncat3: Карты памяти и...,name: Карта памяти 1 ТБ\ncat3: Карты памяти и ...,Товар 1 | name: Карта памяти 64 ГБ\nТовар 2 | ...,(Отличается) Имя : Карта памяти 64 ГБ\n(Отлича...,(Отличается) Имя : Карта памяти 1 ТБ\n(Отличае...
306538,0.0,817327230,822083612,"name: Смартфон Mate48 Pro.. 8/256 ГБ, зеленый\...","name: Смартфон Mate48 Pro.. 10/512 ГБ, зеленый...",Товар 1 | name: Смартфон Mate48 Pro.. 8/256 ГБ...,(Отличается) Имя : Смартфон Mate48 Pro.. 8/256...,(Отличается) Имя : Смартфон Mate48 Pro.. 10/51...


In [ ]:
train = train.drop(["full_text_1", "full_text_2", "text_difference"], axis=1)

In [ ]:
train.to_parquet("train_full_text_w_diff.parquet", index=False)